# Agentic Neurodata Conversion - Jupyter Integration

This notebook demonstrates how to integrate the Agentic Neurodata Conversion MCP Server into a Jupyter research environment.

## Prerequisites

1. Start the MCP server: `pixi run server`
2. Ensure all dependencies are installed: `pixi install`
3. Start Jupyter: `pixi run jupyter notebook`

## Features Demonstrated

- Interactive dataset analysis
- Progress tracking for long operations
- Rich output formatting and visualization
- Error handling and recovery
- Pipeline state management

In [ ]:
# Import required libraries
import json
import time
from pathlib import Path
from typing import Dict, Any, Optional

import requests
from IPython.display import display, HTML, JSON, Markdown
from tqdm.notebook import tqdm

# Configuration
MCP_SERVER_URL = "http://127.0.0.1:8000"
OUTPUT_DIR = "notebook_outputs"

# Create output directory
Path(OUTPUT_DIR).mkdir(exist_ok=True)

print("✓ Setup complete")

## MCP Client Class for Jupyter

This client is optimized for Jupyter notebooks with rich output formatting and progress tracking.

In [ ]:
class JupyterMCPClient:
    """MCP client optimized for Jupyter notebook usage."""
    
    def __init__(self, api_url: str = MCP_SERVER_URL):
        self.api_url = api_url.rstrip("/")
        self.pipeline_state = {}
    
    def _call_tool(self, tool_name: str, payload: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        """Call MCP tool with progress tracking."""
        url = f"{self.api_url}/tool/{tool_name}"
        
        try:
            with tqdm(desc=f"Calling {tool_name}", total=1) as pbar:
                response = requests.post(url, json=payload or {}, timeout=300)
                response.raise_for_status()
                result = response.json()
                pbar.update(1)
                
            return result
        except Exception as e:
            return {"status": "error", "message": str(e)}
    
    def display_result(self, result: Dict[str, Any], title: str = "Result"):
        """Display result with rich formatting."""
        if result.get("status") == "success":
            display(HTML(f"<h3 style='color: green'>✓ {title} - Success</h3>"))
        else:
            display(HTML(f"<h3 style='color: red'>✗ {title} - Failed</h3>"))
            display(HTML(f"<p style='color: red'>{result.get('message', 'Unknown error')}</p>"))
        
        # Display JSON result in collapsible format
        display(JSON(result, expanded=False))
    
    def check_server_status(self):
        """Check server status with rich display."""
        try:
            response = requests.get(f"{self.api_url}/status", timeout=10)
            response.raise_for_status()
            status = response.json()
            
            if status.get("status") == "running":
                display(HTML("<h3 style='color: green'>✓ MCP Server is Running</h3>"))
                display(HTML(f"""
                <table style='border-collapse: collapse; width: 100%'>
                    <tr><td style='border: 1px solid #ddd; padding: 8px'><b>Status</b></td>
                        <td style='border: 1px solid #ddd; padding: 8px'>{status.get('status')}</td></tr>
                    <tr><td style='border: 1px solid #ddd; padding: 8px'><b>Registered Tools</b></td>
                        <td style='border: 1px solid #ddd; padding: 8px'>{status.get('registered_tools', 0)}</td></tr>
                    <tr><td style='border: 1px solid #ddd; padding: 8px'><b>Available Agents</b></td>
                        <td style='border: 1px solid #ddd; padding: 8px'>{", ".join(status.get('agents', []))}</td></tr>
                </table>
                """))
            else:
                display(HTML("<h3 style='color: orange'>⚠ Server Status Unknown</h3>"))
            
            return status
        except Exception as e:
            display(HTML(f"<h3 style='color: red'>✗ Cannot Connect to Server</h3>"))
            display(HTML(f"<p style='color: red'>Error: {e}</p>"))
            display(HTML("<p>Make sure the MCP server is running: <code>pixi run server</code></p>"))
            return {"status": "error", "message": str(e)}

# Create client instance
client = JupyterMCPClient()
print("✓ MCP client created")

## 1. Server Status Check

First, let's verify that the MCP server is running and accessible.

In [ ]:
# Check server status
server_status = client.check_server_status()

## 2. List Available Tools

Let's see what tools are available on the MCP server.

In [ ]:
# Get available tools
try:
    response = requests.get(f"{client.api_url}/tools", timeout=10)
    response.raise_for_status()
    tools = response.json()
    
    display(HTML("<h3>Available MCP Tools</h3>"))
    
    tools_html = "<table style='border-collapse: collapse; width: 100%'>"
    tools_html += "<tr style='background-color: #f2f2f2'><th style='border: 1px solid #ddd; padding: 8px'>Tool Name</th><th style='border: 1px solid #ddd; padding: 8px'>Description</th></tr>"
    
    for tool_name, metadata in tools.get("tools", {}).items():
        description = metadata.get("description", "No description available")
        tools_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'><code>{tool_name}</code></td><td style='border: 1px solid #ddd; padding: 8px'>{description}</td></tr>"
    
    tools_html += "</table>"
    display(HTML(tools_html))
    
except Exception as e:
    display(HTML(f"<p style='color: red'>Error getting tools: {e}</p>"))

## 3. Interactive Dataset Analysis

Let's analyze a dataset interactively. You can modify the dataset path below.

In [ ]:
# Configuration for dataset analysis
DATASET_DIR = "examples/sample-datasets"  # Modify this path as needed
USE_LLM = False  # Set to True to use LLM for enhanced analysis

display(Markdown(f"**Analyzing Dataset:** `{DATASET_DIR}`"))
display(Markdown(f"**Using LLM:** `{USE_LLM}`"))

# Perform analysis
analysis_result = client._call_tool("dataset_analysis", {
    "dataset_dir": DATASET_DIR,
    "use_llm": USE_LLM
})

# Display results
client.display_result(analysis_result, "Dataset Analysis")

# Store metadata for next steps
if analysis_result.get("status") == "success":
    client.pipeline_state["normalized_metadata"] = analysis_result.get("result", {})
    client.pipeline_state["dataset_dir"] = DATASET_DIR
    
    # Show metadata summary
    metadata = client.pipeline_state["normalized_metadata"]
    display(HTML(f"<h4>Metadata Summary</h4>"))
    display(HTML(f"<p><b>Keys extracted:</b> {list(metadata.keys())}</p>"))
    
    if metadata:
        display(JSON(metadata, expanded=True))

## 4. Conversion to NWB Format

Now let's convert the analyzed dataset to NWB format. You'll need to specify the files mapping.

In [ ]:
# Check if we have metadata from previous step
if "normalized_metadata" not in client.pipeline_state:
    display(HTML("<p style='color: red'>No metadata available. Please run the dataset analysis first.</p>"))
else:
    # Configuration for conversion
    FILES_MAP = {
        "recording": "sample_recording.dat",  # Modify as needed
        # Add more file mappings as needed
    }
    
    OUTPUT_NWB_PATH = f"{OUTPUT_DIR}/converted_data.nwb"
    
    display(Markdown(f"**Files Map:** `{FILES_MAP}`"))
    display(Markdown(f"**Output NWB Path:** `{OUTPUT_NWB_PATH}`"))
    
    # Perform conversion
    conversion_result = client._call_tool("conversion_orchestration", {
        "normalized_metadata": client.pipeline_state["normalized_metadata"],
        "files_map": FILES_MAP,
        "output_nwb_path": OUTPUT_NWB_PATH
    })
    
    # Display results
    client.display_result(conversion_result, "NWB Conversion")
    
    # Store NWB path for next steps
    if conversion_result.get("status") == "success":
        client.pipeline_state["nwb_path"] = conversion_result.get("output_nwb_path")
        display(HTML(f"<p><b>NWB file created:</b> <code>{client.pipeline_state['nwb_path']}</code></p>"))

## 5. NWB File Evaluation

Let's evaluate the quality of the generated NWB file.

In [ ]:
# Check if we have an NWB file from previous step
if "nwb_path" not in client.pipeline_state:
    display(HTML("<p style='color: red'>No NWB file available. Please run the conversion first.</p>"))
else:
    nwb_path = client.pipeline_state["nwb_path"]
    
    display(Markdown(f"**Evaluating NWB File:** `{nwb_path}`"))
    
    # Check if file exists
    if not Path(nwb_path).exists():
        display(HTML(f"<p style='color: red'>NWB file not found: {nwb_path}</p>"))
    else:
        # Perform evaluation
        evaluation_result = client._call_tool("evaluate_nwb_file", {
            "nwb_path": nwb_path,
            "generate_report": True
        })
        
        # Display results
        client.display_result(evaluation_result, "NWB Evaluation")
        
        # Show evaluation summary
        if evaluation_result.get("status") == "success":
            evaluation = evaluation_result.get("result", {})
            
            # Create summary table
            summary_html = "<h4>Evaluation Summary</h4>"
            summary_html += "<table style='border-collapse: collapse; width: 100%'>"
            
            if "validation_errors" in evaluation:
                error_count = len(evaluation["validation_errors"])
                color = "red" if error_count > 0 else "green"
                summary_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'><b>Validation Errors</b></td><td style='border: 1px solid #ddd; padding: 8px; color: {color}'>{error_count}</td></tr>"
            
            if "quality_score" in evaluation:
                score = evaluation["quality_score"]
                color = "green" if score > 0.8 else "orange" if score > 0.6 else "red"
                summary_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'><b>Quality Score</b></td><td style='border: 1px solid #ddd; padding: 8px; color: {color}'>{score:.2f}</td></tr>"
            
            summary_html += "</table>"
            display(HTML(summary_html))
            
            # Show first few validation errors if any
            if "validation_errors" in evaluation and evaluation["validation_errors"]:
                errors = evaluation["validation_errors"][:5]  # Show first 5
                errors_html = "<h4>Validation Errors (first 5)</h4><ul>"
                for error in errors:
                    errors_html += f"<li style='color: red'>{error}</li>"
                errors_html += "</ul>"
                if len(evaluation["validation_errors"]) > 5:
                    errors_html += f"<p><i>... and {len(evaluation['validation_errors']) - 5} more errors</i></p>"
                display(HTML(errors_html))

## 6. Knowledge Graph Generation

Finally, let's generate a knowledge graph from the NWB data.

In [ ]:
# Check if we have an NWB file
if "nwb_path" not in client.pipeline_state:
    display(HTML("<p style='color: red'>No NWB file available. Please run the conversion first.</p>"))
else:
    nwb_path = client.pipeline_state["nwb_path"]
    
    display(Markdown(f"**Generating Knowledge Graph from:** `{nwb_path}`"))
    
    # Check if file exists
    if not Path(nwb_path).exists():
        display(HTML(f"<p style='color: red'>NWB file not found: {nwb_path}</p>"))
    else:
        # Perform knowledge graph generation
        kg_result = client._call_tool("generate_knowledge_graph", {
            "nwb_path": nwb_path,
            "output_format": "ttl",
            "include_provenance": True
        })
        
        # Display results
        client.display_result(kg_result, "Knowledge Graph Generation")
        
        # Show knowledge graph summary
        if kg_result.get("status") == "success":
            kg_info = kg_result.get("result", {})
            
            summary_html = "<h4>Knowledge Graph Summary</h4>"
            summary_html += "<table style='border-collapse: collapse; width: 100%'>"
            
            if "graph_path" in kg_info:
                summary_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'><b>Graph File</b></td><td style='border: 1px solid #ddd; padding: 8px'><code>{kg_info['graph_path']}</code></td></tr>"
            
            if "triple_count" in kg_info:
                summary_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'><b>Triples Generated</b></td><td style='border: 1px solid #ddd; padding: 8px'>{kg_info['triple_count']:,}</td></tr>"
            
            if "format" in kg_info:
                summary_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'><b>Format</b></td><td style='border: 1px solid #ddd; padding: 8px'>{kg_info['format']}</td></tr>"
            
            summary_html += "</table>"
            display(HTML(summary_html))

## 7. Pipeline Summary

Let's review what we've accomplished in this session.

In [ ]:
# Display pipeline summary
display(HTML("<h3>Pipeline Summary</h3>"))

summary_html = "<table style='border-collapse: collapse; width: 100%'>"
summary_html += "<tr style='background-color: #f2f2f2'><th style='border: 1px solid #ddd; padding: 8px'>Step</th><th style='border: 1px solid #ddd; padding: 8px'>Status</th><th style='border: 1px solid #ddd; padding: 8px'>Details</th></tr>"

# Dataset Analysis
if "normalized_metadata" in client.pipeline_state:
    summary_html += "<tr><td style='border: 1px solid #ddd; padding: 8px'>Dataset Analysis</td><td style='border: 1px solid #ddd; padding: 8px; color: green'>✓ Complete</td><td style='border: 1px solid #ddd; padding: 8px'>Metadata extracted</td></tr>"
else:
    summary_html += "<tr><td style='border: 1px solid #ddd; padding: 8px'>Dataset Analysis</td><td style='border: 1px solid #ddd; padding: 8px; color: red'>✗ Not completed</td><td style='border: 1px solid #ddd; padding: 8px'>-</td></tr>"

# NWB Conversion
if "nwb_path" in client.pipeline_state:
    nwb_exists = Path(client.pipeline_state["nwb_path"]).exists()
    if nwb_exists:
        summary_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'>NWB Conversion</td><td style='border: 1px solid #ddd; padding: 8px; color: green'>✓ Complete</td><td style='border: 1px solid #ddd; padding: 8px'><code>{client.pipeline_state['nwb_path']}</code></td></tr>"
    else:
        summary_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px'>NWB Conversion</td><td style='border: 1px solid #ddd; padding: 8px; color: orange'>⚠ File missing</td><td style='border: 1px solid #ddd; padding: 8px'><code>{client.pipeline_state['nwb_path']}</code></td></tr>"
else:
    summary_html += "<tr><td style='border: 1px solid #ddd; padding: 8px'>NWB Conversion</td><td style='border: 1px solid #ddd; padding: 8px; color: red'>✗ Not completed</td><td style='border: 1px solid #ddd; padding: 8px'>-</td></tr>"

summary_html += "</table>"
display(HTML(summary_html))

# Show pipeline state
if client.pipeline_state:
    display(HTML("<h4>Current Pipeline State</h4>"))
    display(JSON(client.pipeline_state, expanded=False))

# Save session results
session_file = f"{OUTPUT_DIR}/jupyter_session_results.json"
with open(session_file, "w") as f:
    json.dump(client.pipeline_state, f, indent=2, default=str)

display(HTML(f"<p><b>Session results saved to:</b> <code>{session_file}</code></p>"))

## 8. Next Steps

Here are some suggestions for further exploration:

1. **Modify Parameters**: Try different datasets, enable LLM analysis, or adjust conversion parameters
2. **Error Recovery**: Experiment with error handling and recovery mechanisms
3. **Batch Processing**: Process multiple datasets in a loop
4. **Custom Analysis**: Add your own analysis functions that work with the MCP server results
5. **Visualization**: Create plots and visualizations of the conversion results

## Troubleshooting

If you encounter issues:

1. **Server Connection**: Ensure the MCP server is running (`pixi run server`)
2. **File Paths**: Verify that dataset and file paths are correct
3. **Dependencies**: Make sure all required packages are installed (`pixi install`)
4. **Logs**: Check the MCP server logs for detailed error messages

For more information, see the documentation in the `examples/` directory.